# Taller de Investigación
## Trabajando con la simulación semianalitica mini

La idea inicial, lectura de archivos, calculo de distrancia entre halos e identificación de las caracteristicas del ambiente. 

In [1]:
import numpy as np
import h5py # que pasa con hdf5? 
from collections import Counter

### Lectura de archivos
Primero la lectura de datos: A continuación solo llama columnas de ID, X, Y, Z (trabajo con la pato)

In [2]:
datos = np.genfromtxt("DATA/sussing_125.z0.000.AHF_halos", usecols=[0,5,6,7], names=True)
#datos

Clase de lectura de archivos, donde se requieren las columnas iniciales de interes de los archis, se lee el tipo hdf5 y se guarda la información asociada a un halo en particular. 

In [3]:
class read_file: 
    
    def __init__(self, path, file, columns=None):
        if columns is not None:
            self.cols = columns
            #self.hostcol = self.cols[1]; self.Numcol = self.cols[2]; self.Mcol = self.cols[3]; 
        else:
            self.cols     = [0, 1, 2, 3, 5, 6, 7] #ID, hosthalo, substructuras, masa y posiciones xyz
            self.hostcol  = self.cols[1]; self.Numcol = self.cols[2]; self.Mcol = self.cols[3]; 
            
        self.path = path; self.file = file; self.read()
        
    def read(self):
        if self.file == "*.hdf5":
            print("No se ha hecho esta parte")
        else:
            data = np.genfromtxt(str(self.path+self.file), usecols = self.cols, names=True)
            self.data = np.array([list(row) for row in data])
            
    def halo_info(self, index, info=False): 
        self.hinfo = self.data[index]
        
        return(self.hinfo)

Ejemplo de lectura de archivos

In [4]:
sussing_125 = read_file("DATA/", "sussing_125.z0.000.AHF_halos")
a = sussing_125.halo_info(10)

Ahora se realizan calculos de distancia y velocidades. 

In [7]:
class kinetic: #cinematica entre dos halos
    
    def __init__(self, halo1, halo2, position_cols, velocity_cols=None):
        self.haloA = halo1; self.haloB=halo2
        position(); distance()

    def position(self, position_cols): #positions cols: lista correspondiente a las columnas x, y, z.
        X = position_cols[0]; Y = position_cols[1]; Z = position_cols[3] 
        self.position = np.array([X,Y,Z])

    def distance(self, posA, posB): 
        return(np.linalg.norm(posA-posB))
    
    #def velocity(self, , halo2): 
        #return(np.linalg.norm(halo1-halo2))

In [ ]:
distancia = kinetic()

### Tarea I: gráfica de halos vs masa

In [ ]:
sussing_125.data[]